In [212]:
import numpy as np
import random
import math

from util import read_nxgraph
from util import obj_maxcut

In [213]:
class Bucket:
    def __init__(self, gmin, gmax, solution, graph):
        self.gmin = gmin
        self.gmax = gmax
        self.graph = graph
        self.solution = solution
        self.numBuckets = gmax - gmin + 1

        self.bucketsA = [[] for _ in range(self.numBuckets)]
        self.bucketsB = [[] for _ in range(self.numBuckets)]

        self.maxgainA = None
        self.maxgainB = None

        self.vertexMap = {}

    def calcGain(self, vertex):
        newGraph = self.get_combined_buckets()
        vertexVal = newGraph[vertex]
        if vertexVal == 0:
            newGraph[vertex] = 1
        else:
            newGraph[vertex] = 0

        return math.ceil(obj_maxcut(newGraph, self.graph) - obj_maxcut(self.get_combined_buckets(), self.graph))
    
    def calcInitial(self, vertex):
        newGraph = self.solution
        vertexVal = newGraph[vertex]
        if vertexVal == 0:
            newGraph[vertex] = 1
        else:
            newGraph[vertex] = 0

        return math.ceil(obj_maxcut(newGraph, self.graph) - obj_maxcut(self.solution, self.graph))

    def addInitial(self, vertex, partition):
        gain = self.calcInitial(vertex) # Get gain value from vertex i
        bucketArray = self.bucketsA if partition == 1 else self.bucketsB

        bucketArray[gain].append(vertex) # Add vertex i to bucket assigned by gain
        self.vertexMap[vertex] = (partition, gain, vertex) # Store partition, gain value, vertex i in Map
        self.updateMaxGain(partition, vertex)

    # Add vertex to proper bucket
    def addVertex(self, vertex, partition):
        gain = self.calcGain(vertex) # Get gain value from vertex i
        bucketArray = self.bucketsA if partition == 1 else self.bucketsB

        bucketArray[gain].append(vertex) # Add vertex i to bucket assigned by gain
        self.vertexMap[vertex] = (partition, gain, vertex) # Store partition, gain value, vertex i in Map
        self.updateMaxGain(partition, vertex)

    # Remove vertex from its bucket
    def removeVertex(self, vertex):
        if vertex in self.vertexMap:
            partition, index, _ = self.vertexMap[vertex]
            bucket_array = self.bucketsA if partition == 1 else self.bucketsB

            bucket_array[index].remove(vertex)
            del self.vertexMap[vertex]
            self.updateMaxGain(partition)

    # Update gain of vertex given
    def updateGain(self, vertex, new_gain):
        if vertex in self.vertexMap:
            partition, _, _ = self.vertexMap[vertex]
            self.removeVertex(vertex)
            self.addVertex(vertex, partition)

    # Move vertex from one partition to the other
    def moveVertex(self, vertex):
        if vertex in self.vertexMap:
            partition, _, _ = self.vertexMap[vertex]
            self.removeVertex(vertex)
            new_partition = 0 if partition == 1 else 1
            self.addVertex(vertex, new_partition)

    def updateMaxGain(self, partition, start_index=None):
        bucket_array = self.bucketsA if partition == 1 else self.bucketsB
        maxgain_attr = 'maxgainA' if partition == 1 else 'maxgainB'

        if start_index is None:
            for i in range(self.numBuckets - 1, -1, -1):
                if bucket_array[i]:
                    setattr(self, maxgain_attr, i)
                    return
            setattr(self, maxgain_attr, None)
        elif bucket_array[start_index]:
            current_max = getattr(self, maxgain_attr)
            setattr(self, maxgain_attr, max(current_max or 0, start_index))
        else:
            self.updateMaxGain(partition)

    def get_best_move(self):
        bestA = None if self.maxgainA is None else self.bucketsA[self.maxgainA][-1]
        bestB = None if self.maxgainB is None else self.bucketsB[self.maxgainB][-1]

        return bestA if self.maxgainA >= self.maxgainB else bestB
    
    def getBestMoveA(self):
        bestA = None if self.maxgainA is None else self.bucketsA[self.maxgainA][-1]

        return bestA
    
    def getBestMoveB(self):
        bestB = None if self.maxgainB is None else self.bucketsB[self.maxgainB][-1]

        return bestB
    
    def get_combined_buckets(self):
            combined_buckets = []
            for i in range(self.numBuckets):
                combined_buckets.extend(self.bucketsA[i])
                combined_buckets.extend(self.bucketsB[i])
            return combined_buckets


In [214]:
graph = read_nxgraph('.././data/gset/gset_14.txt')
currSolution = list(np.random.randint(0, 2, graph.number_of_nodes()))
bucket = Bucket(graph.number_of_nodes() * -1, graph.number_of_nodes(), currSolution, graph)
for i in range(graph.number_of_nodes()):
    if(currSolution[i] == 0):
        bucket.addInitial(i, 0)
    else:
        bucket.addInitial(i, 1)

currSolution = bucket
currVal = obj_maxcut(currSolution.get_combined_buckets(), graph)
bestSolution = currSolution
bestVal = currVal
prevSolution = currSolution

In [215]:
omega = 0
iteration = 0
L0 = int(0.01*graph.number_of_nodes())
T = 1000
gamma = random.randint(3,(graph.number_of_nodes()/10))
P0 = 0.8
Q = 0.5
H = {}

In [216]:
def isNotTabu(H, vertex, gamma, iteration):
    if vertex in H:
        if((H[vertex]+gamma) < iteration):
            return True
    return False

In [217]:
def perturb(C,L,M,iteration,H):
    for _ in range(L):
        bestVertex = C.get_best_move() - 1
        bestVertexA = C.getBestMoveA() - 1
        bestVertexB = C.getBestMoveB() - 1
        if(M == 'A2' and (isNotTabu(H, bestVertexA, gamma, iteration) and isNotTabu(H, bestVertexB, gamma, iteration))):
            C.moveVertex(bestVertexA)
            C.moveVertex(bestVertexB)
            H[bestVertexA] = iteration + gamma
            H[bestVertexB] = iteration + gamma
        elif(M == 'A1' and isNotTabu(H, bestVertex, gamma, iteration)):
            C.moveVertex(bestVertex)
            H[bestVertex] = iteration + gamma
        else:
            randomVertex = random.randint(0, len(C.get_combined_buckets())-2)
            C.moveVertex(randomVertex)
            H[randomVertex] = iteration + gamma
            
            
        
        iteration = iteration + 1
    
    return C, H, iteration

In [218]:
def perturbation(C, L, H, iteration, omega):
    if omega == 0:
        C, H, iteration = perturb(C,L,'B')
    else:
        if (np.exp(-(omega)/T) > P0):
            P = np.exp(-(omega)/T)
        else:
            P = P0

        randProb = random.random()

        if (randProb < (P * Q)):
            C, H, iteration = perturb(C,L,'A1',iteration,H)
        elif (randProb < (P * Q + P * (1-Q))):
            C, H, iteration = perturb(C,L,'A2',iteration,H)
        else:
            C, H, iteration = perturb(C,L,'B',iteration,H)

    return C, H, iteration

In [221]:
while(iteration < 200000*graph.number_of_nodes()):
    vertexToMove = currSolution.get_best_move()
    combo = currSolution
    combo.moveVertex(vertexToMove-1)
    while(obj_maxcut(combo.get_combined_buckets(), graph) > obj_maxcut(currSolution.get_combined_buckets(), graph)):
        currVal = obj_maxcut(combo, graph)
        currSolution = combo
        H[vertexToMove-1] = iteration + gamma 
        iteration = iteration + 1
    
    if currVal > bestVal:
        bestSolution = currSolution
        bestVal = currVal
        omega = 0
    else:
        omega = omega + 1

    # Determine the number of perturbation moves L to be applied to C

    if omega > T:
        # Search is stagnating, random perturbation is required
        omega = 0

    if currSolution == prevSolution:
        L = int(L + 1)
    else:
        L = int(L0)
    
    prevSolution = currSolution
    currSolution, H, iteration = perturbation(currSolution, L, H, iteration, omega)
    print(iteration)

338
509
681
854
1028
1203
1379
1556
1734
1913
2093
2274
2456
2639
2823
3008
3194
3381
3569
3758
3948
4139
4331
4524
4718
4913
5109
5306
5504
5703
5903
6104
6306
6509
6713
6918
7124
7331
7539
7748
7958
8169
8381
8594
8808
9023
9239
9456
9674
9893
10113
10334
10556
10779
11003
11228
11454
11681
11909
12138
12368
12599
12831
13064
13298
13533
13769
14006
14244
14483
14723
14964
15206
15449
15693
15938
16184
16431
16679
16928
17178
17429
17681
17934
18188
18443
18699
18956
19214
19473
19733
19994
20256
20519
20783
21048
21314
21581
21849
22118
22388
22659
22931
23204
23478
23753
24029
24306
24584
24863
25143
25424
25706
25989
26273
26558
26844
27131
27419
27708
27998
28289
28581
28874
29168
29463
29759
30056
30354
30653
30953
31254
31556
31859
32163
32468
32774
33081
33389
33698
34008
34319
34631
34944
35258
35573
35889
36206
36524
36843
37163
37484
37806
38129
38453
38778
39104
39431
39759
40088
40418
40749
41081
41414
41748
42083
42419
42756
43094
43433
43773
44114
44456
44799
45143
4548

KeyboardInterrupt: 

In [222]:
print(currVal)

4694.0
